In [ ]:
import cloudscraper
import re
import json
import numpy as np
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import sys
sys.path.insert(0, '../data_prep')
from sqlite_utils import get_from_table, table_to_df
from tmdbv3api import Person

In [4]:
film_id = 'f_0l5xU'
film_title = get_from_table('FILM_TITLE', film_id, 'FILM_TITLE')
letterboxd_url = get_from_table('FILM_TITLE', film_id, 'LETTERBOXD_URL')
print('Film ID is "{}", film title is "{}", and url is {}'.format(film_id, film_title, letterboxd_url))

Film ID is "f_0l5xU", film title is "Waltzing with Brando", and url is https://boxd.it/l5xU


In [5]:
letterboxd_url = get_from_table('FILM_TITLE', film_id, 'LETTERBOXD_URL')
scraper = cloudscraper.create_scraper()
r = scraper.get(letterboxd_url)
redirected_url = r.url
if letterboxd_url != redirected_url:
    r = scraper.get(redirected_url)
soup = BeautifulSoup(r.content, 'lxml')
rating_dict = json.loads(soup.find('script', {'type':"application/ld+json"}).string.split('\n')[2]).get('aggregateRating')
try:
    rating_mean = rating_dict.get('ratingValue')
except:
    rating_mean = np.nan
try:
    rating_count = rating_dict.get('ratingCount')
except:
    rating_count = np.nan

In [6]:
film_url_title = get_from_table('FILM_URL_TITLE', film_id, 'FILM_URL_TITLE')
initial_url = 'https://letterboxd.com/film/{}/members/rated/.5-5/'.format(film_url_title)
scraper = cloudscraper.create_scraper()
r = scraper.get(initial_url)
redirected_url = r.url
if initial_url != redirected_url:
    new_film_url_title = redirected_url.replace('https://letterboxd.com/film/', '').replace('/members/rated/.5-5/', '')
    film_url_title = new_film_url_title
soup = BeautifulSoup(r.content, 'lxml')
metrics_dict = {}
for i in ['members', 'fans', 'likes', 'reviews', 'lists']:
    href_str = '/film/{}/{}/'.format(film_url_title, i)
    try:
        metric_string = soup.find('a', {'class': 'tooltip', 'href':href_str}).get('title')
        metric = int(metric_string[:metric_string.find('\xa0')].replace(',', ''))
    except:
        metric = 0
    metrics_dict[i] = metric
metrics_dict

{'members': 155, 'fans': 0, 'likes': 21, 'reviews': 44, 'lists': 477}

In [7]:
json.loads(soup.find('script', {'type':"application/ld+json"}).string.split('\n')[2]).keys()

AttributeError: 'NoneType' object has no attribute 'string'

In [45]:
print(str(soup))

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]--><!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]--><!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>Access denied | letterboxd.com used Cloudflare to restrict access</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="noindex, nofollow" name="robots"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="/cdn-cgi/styles/main.css" id="cf_styles-css" rel="stylesheet"/>
<script>
(function(){if(document.addEventListener&&window.XMLHttpRequest&&JSON&&JSON.stringify){var e=function(a){var c=document.getElementById("error-feedback-survey"),d=document.getElementById("error-feedback-success"),b=new XMLHttpRequest;a={event:"feedba

In [8]:
scraper = cloudscraper.create_scraper()
r = scraper.get('https://letterboxd.com/film/{}/'.format(film_url_title))
soup = BeautifulSoup(r.content, 'lxml')

In [9]:
import re
[str(soup)[m.start()-20: m.start()+100] for m in re.finditer('esi', str(soup))]

['tps://a.ltrbxd.com/resized/sm/upload/4w/kk/98/d3/8uk1sUuiKJtnCPT2Dd2nu58ywhD-1200-1200-675-675-crop-000000.jpg?v=e67c7e8',
 'tps://a.ltrbxd.com/resized/sm/upload/4w/kk/98/d3/8uk1sUuiKJtnCPT2Dd2nu58ywhD-1200-1200-675-675-crop-000000.jpg?v=e67c7e8',
 'tps://a.ltrbxd.com/resized/sm/upload/4w/kk/98/d3/8uk1sUuiKJtnCPT2Dd2nu58ywhD-1200-1200-675-675-crop-000000.jpg?v=e67c7e8',
 'tps://a.ltrbxd.com/resized/sm/upload/4w/kk/98/d3/8uk1sUuiKJtnCPT2Dd2nu58ywhD-960-960-540-540-crop-000000.jpg?v=e67c7e851',
 'rando/availability/?esiAllowUser=true&amp;esiAllowCountry=true">\n<section class="watch-panel">\n<div class="header">\n<h3 ',
 'iAllowUser=true&amp;esiAllowCountry=true">\n<section class="watch-panel">\n<div class="header">\n<h3 class="title">Where to',
 'debar-user-actions/?esiAllowUser=true">\n</span>\n<script crossorigin="" src="https://s.ltrbxd.com/static/js/es/csi/sideba',
 'do/friend-activity/?esiAllowUser=true">\n</div>\n<div class="csi js-csi" data-on-load="production-news" data-src="/

In [10]:
str(soup)[1067:1100]

'esized/sm/upload/4w/kk/98/d3/8uk1'

In [11]:
soup.find('a', {'class': 'has-icon icon-top250 icon-16 tooltip'})

In [13]:
scraper = cloudscraper.create_scraper()
r = scraper.get(letterboxd_url)
redirected_url = r.url
if letterboxd_url != redirected_url:
    r = scraper.get(redirected_url)    
soup = BeautifulSoup(r.content, 'lxml')
cast = soup.find('div', {'id': 'tab-cast'}).findAll('a')
cast_record = {
    'FILM_ID': [film_id]*len(cast),
    'PERSON_ID': [Person().search(x.text)[0].get('id') for x in cast],
    'CHARACTER': [x.get('title') for x in cast],
    'CAST_ORDER': range(len(cast)),
    'CREATED_AT': [datetime.now()]*len(cast)
    }

pd.DataFrame(cast_record)

,FILM_ID,PERSON_ID,CHARACTER,CAST_ORDER,CREATED_AT
0,f_0l5xU,1954,Marlon Brando,0,2025-06-25 10:51:41.768377
1,f_0l5xU,53926,Bernard Judge,1,2025-06-25 10:51:41.768377
2,f_0l5xU,1460731,Michelle,2,2025-06-25 10:51:41.768377
3,f_0l5xU,169725,Dana,3,2025-06-25 10:51:41.768377
4,f_0l5xU,52997,Jack Bellin,4,2025-06-25 10:51:41.768377
5,f_0l5xU,13445,Madame Leroy,5,2025-06-25 10:51:41.768377
6,f_0l5xU,3037,Seymour Kraft,6,2025-06-25 10:51:41.768377
7,f_0l5xU,1118978,Zeke Knight,7,2025-06-25 10:51:41.768377
8,f_0l5xU,3210443,Maria Schneider,8,2025-06-25 10:51:41.768377
9,f_0l5xU,4623528,Nick Rutgers,9,2025-06-25 10:51:41.768377


In [16]:
crew = soup.find('div', {'id': 'tab-crew'}).findAll('a')
crew

[<a class="text-slug" href="/director/bill-fishman/">Bill Fishman</a>,
 <a class="text-slug" href="/producer/billy-zane/">Billy Zane</a>,
 <a class="text-slug" href="/producer/bill-fishman/">Bill Fishman</a>,
 <a class="text-slug" href="/producer/dean-bloxom/">Dean Bloxom</a>,
 <a class="text-slug" href="/writer/bill-fishman/">Bill Fishman</a>,
 <a class="text-slug" href="/story/bernard-judge/">Bernard Judge</a>,
 <a class="text-slug" href="/editor/michael-yanovich/">Michael Yanovich</a>,
 <a class="text-slug" href="/cinematography/garrett-obrien/">Garrett O'Brien</a>,
 <a class="text-slug" href="/composer/matei-bratescot/">Matei Bratescot</a>,
 <a class="text-slug" href="/sound/clement-mattox/">Clement Mattox</a>]

In [24]:
jobs = [x.get('href').split('/')[1] for x in crew]
names = [x.get('href').split('/')[2] for x in crew]
print('\n'.join([j+': ' + n for j, n in zip(jobs, names)]))


director: bill-fishman
producer: billy-zane
producer: bill-fishman
producer: dean-bloxom
writer: bill-fishman
story: bernard-judge
editor: michael-yanovich
cinematography: garrett-obrien
composer: matei-bratescot
sound: clement-mattox


In [27]:
r = scraper.get('https://letterboxd.com/film/{}/members/rated/.5-5/'.format(film_url_title))
r

<Response [200]>

In [28]:
soup = BeautifulSoup(r.content, 'lxml')
soup

<!DOCTYPE html>
<html class="context-client-unknown no-mobile no-js" id="html" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=1024" name="viewport"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="Everyone who has watched Waltzing with Brando" name="description"/>
<meta content="https://letterboxd.com/film/waltzing-with-brando/members/rated/.5-5/" property="og:url"/>
<meta content="Everyone who has watched Waltzing with Brando" property="og:title"/>
<meta content="Everyone who has watched Waltzing with Brando" property="og:description"/>
<meta content="https://s.ltrbxd.com/static/img/default-share-7md9L34t.png" property="og:image"/>
<meta content="173683136069040" property="fb:app_id"/>
<meta content="Letterboxd" name="application-name"/>
<meta content="#14181C" name="theme-color"/>
<meta content="#445566" name="msapplication-TileColor"/>
<meta content="app-id=1054271011, affiliate-data=11l5KW, app-argument=https://letterboxd.com/film/wal

In [30]:
metrics_dict = {}
for i in ['members', 'fans', 'likes', 'reviews', 'lists']:
    href_str = '/film/{}/{}/'.format(film_url_title, i)
    try:
        metric_string = soup.find('a', {'class': 'tooltip', 'href':href_str}).get('title')
        metric = int(metric_string[:metric_string.find('\xa0')].replace(',', ''))
    except:
        metric = 0
    metrics_dict[i] = metric
metrics_dict

{'members': 155, 'fans': 0, 'likes': 21, 'reviews': 44, 'lists': 477}

In [32]:
film_url_title = get_from_table('FILM_URL_TITLE', film_id, 'FILM_URL_TITLE')
scraper = cloudscraper.create_scraper()
r = scraper.get('https://letterboxd.com/esi/film/{}/stats/'.format(film_url_title))
soup = BeautifulSoup(r.content, 'lxml')
try:
    top_250_status = int(soup.find('a', {'class': 'has-icon icon-top250 icon-16 tooltip'}).text)
except:
    top_250_status = None
top_250_status